# Relief系列算法总结
&emsp;&emsp;在机器学习的实际应用中，特征数量可能较多，其中可能存在不相关的特征，特征之间也可能存在相关性，容易导致如下的后果：
1. 特征个数越多，分析特征、训练模型所需的时间就越长，模型也会越复杂。
2. 特征个数越多，容易引起“维度灾难”，其推广能力会下降。
3. 特征个数越多，容易导致机器学习中经常出现的特征稀疏的问题，导致模型效果下降。
4. 对于模型来说，可能会导致不适定的情况，即是解出的参数会因为样本的微小变化而出现大的波动。<br>

特征选择，能剔除不相关、冗余、没有差异刻画能力的特征，从而达到减少特征个数、减少训练或者运行时间、提高模型精确度的作用。
Relief是一系列的特征权重算法，包括Relief（二分类），ReliefF（多分类）以及RReliefF（目标属性为连续值的回归问题）。本文仅介绍前两者。


## Relief思想
&emsp;&emsp;根据各个特征和类别的相关性赋予特征不同的权重，权重小于某个阈值的特征将被移除。

## 原理
&emsp;&emsp;Relief算法中特征和类别的相关性是基于特征对近距离样本的区分能力。
### Relief算法
&emsp;&emsp;算法从训练集D中随机选择一个样本R，然后从和R同类的样本中寻找最近邻样本H，称为Near Hit，从和R不同类的样本中寻找最近邻样本M，称为NearMiss，然后根据以下规则更新每个特征的权重：如果R和Near Hit在某个特征上的距离小于R和Near Miss上的距离，则说明该特征对区分同类和不同类的最近邻是有益的，则增加该特征的权重；反之，如果R和Near Hit在某个特征的距离大于R和Near Miss上的距离，说明该特征对区分同类和不同类的最近邻起负面作用，则降低该特征的权重。以上过程重复m次，最后得到各特征的平均权重。特征的权重越大，表示该特征的分类能力越强，反之，表示该特征分类能力越弱。<br>

&emsp;&emsp;假设一个样例X有p个特征，S为样本量为n的训练样本集，F即${f_{1},f_{2},...f_{p}}$为特征集，一个样例X由p维向量$(x_{1},x_{2},...x_{p})$构成，其中，$x_{j}$为X的第j个特征的值。
&emsp;&emsp;relief算法可以解决名义变量和数值变量，两个样例X和Y的特征的值的差可由下面的函数定义：

&emsp;&emsp;当$x_{k}$和$y_{k}$为名义变量时：
$$diff(x_{k},y_{k})= \begin{cases}
0,如果x_{k}和y_{k}相同\\
1,如果x_{k}和y_{k}不同\\
\end{cases}$$

&emsp;&emsp;当$x_{k}$和$y_{k}$为数值变量时：
$$diff(x_{k},y_{k})= (x_{k}-y_{k})/v_{k}$$
$v_{k}$为归一化单位，把diff值归一到[0,1]区间，可以在之前先把数值变量进行归一化。

&emsp;&emsp;<font color = red size = 4>算法流程如下：</font><br>

输入：样本集S，抽样次数m，特征权重阈值$\tau$<br>
输出：选择后的特征集
+ 把S分成$S^{+}={正例}$和$S^{-}={负例}$
+ 权重W=(0,0,...0)
+ For i = 1 to m
    * 随机选择一个样例$X \in S$
    * 随机选择一个距离X最近邻的正例$Z^{+} \in S^{+}$
    * 随机选择一个距离X最近邻的负例$Z^{-} \in S^{-}$
    * if X是一个正例
        - then Near_hit = $Z^{+}$; Near_miss = $Z^{-}$
    * else Near_hit = $Z^{-}$; Near_miss = $Z^{+}$
    * for i = 1 to p
        - $W_{i} = W_{i} - diff(x_{i},near\_hit_{i})^{2} + diff(x_{i},near\_miss_{i})^{2}$

+ relevance = $\frac{W}{m}$
+ for i = 1 to p
    * if $relevance_{i} \geq \tau$
        - then $f_{i}$是一个相关的特征
    * else $f_{i}$是一个不相关的特征 
        
&emsp;&emsp;relief在下列情况有效：（1）相关性水平对于相关的特征很大，对于不相关的特征很小，（2）$\tau$用来选择相关特征，去除不相关特征。<br> 
&emsp;&emsp;计算复杂度：$\Theta(pmn)$，p为特征数，m为迭代次数，n为样例数。可以看出该算法的运行时间随着样本的抽样次数m和原始特征个数p的增加线性增加，因此运行效率非常高。<br>

### ReliefF算法
&emsp;&emsp;ReliefF算法在处理多类问题时，每次从训练样本集中随机取出一个样本R，然后从和R同类的样本集中找出R的k个近邻样本(near Hits)，从每个R的不同类的样本集中均找出k个近邻样本(near Misses)，然后更新每个特征的权重，如下式所示：
$$W(A)=W(A)-\sum_{j=1}^{k}\frac{A,R,H_{j}}{mk}+\sum_{C \notin class(R)}[\frac{p(C)}{1-p(class(R))} \sum_{j=1}^{k}diff(A,R,M_{j}(C))]/(mk)$$

&emsp;&emsp;上式中，$diff(A,R_{1},R_{2})$表示样本$R_{1}$和$R_{2}$在特征A上的差，$M_{j}(C)$表示类$C \notin class(R)$中第j个最近邻样本。如下式：
$$diff(A,R_{1},R_{2}) = \begin{cases}
\frac{|R_{1}[A]-R_{2}[A]|}{max(A)-min(A)},如果A是连续的\\
\begin{cases}
0,如果A是离散的，且 R_{1}[A]=R_{2}[A]\\
1,如果A是离散的，且 R_{1}[A]\neq R_{2}[A]\\
\end{cases}
\end{cases}$$

&emsp;&emsp;<font color = red size = 4>算法流程如下：</font><br>
输入：训练集D，抽样次数m，特征权重阈值$\tau$，最近邻样本个数k
输出：各个特征的特征权重T
+ 置所有特征权重为0，T为空集
+ for i = 1 to m
    * 从D中随机选择一个样本R
    * 从R的同类样本集中找到R的k个最近邻$H_{j}(j = 1,2,...,k)$，从每个不同类样本集中找到k个最近邻$M_{j}(C)$;
+ for A = 1 to N (all features)
    * $W(A) = W(A)-\sum_{j=1}^{k}diff(A,R,H_{j})/(mk)+\sum_{C \notin class(R)}[\frac{p(C)}{1-p(class(R))} \sum_{j=1}^{k}diff(A,R,M_{j}(C))]/(mk)$


## python实现

In [11]:
import pandas as pd
import numpy as np
from sklearn.pipeline import make_pipeline
from skrebate import ReliefF
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

genetic_data = pd.read_csv('GAMETES_Epistasis_2-Way_20atts_0.4H_EDM-1_1.txt',
                           sep='\t')
print genetic_data.head()
features, labels = genetic_data.drop('class', axis=1).values, genetic_data['class'].values

clf = make_pipeline(ReliefF(n_features_to_select=2, n_neighbors=100),
                    RandomForestClassifier(n_estimators=100))

print(np.mean(cross_val_score(clf, features, labels)))

   N0  N1  N2  N3  N4  N5  N6  N7  N8  N9  ...    N11  N12  N13  N14  N15  \
0   0   0   2   1   1   0   0   2   0   1  ...      2    1    0    0    0   
1   0   0   1   0   0   1   1   0   0   1  ...      0    1    0    0    0   
2   0   0   0   1   0   2   0   0   0   0  ...      1    1    0    0    2   
3   0   1   0   2   0   1   0   2   0   0  ...      1    1    0    1    1   
4   0   0   1   1   0   0   0   1   1   0  ...      1    0    0    1    0   

   N16  N17  P1  P2  class  
0    0    0   0   1      1  
1    0    0   0   0      1  
2    2    0   0   0      1  
3    0    1   1   0      1  
4    0    1   1   0      1  

[5 rows x 21 columns]
0.793125134935


In [16]:
from sklearn.model_selection import train_test_split

# Make sure to compute the feature importance scores from only your training set
X_train, X_test, y_train, y_test = train_test_split(features, labels)

fs = ReliefF()
fs.fit(X_train, y_train)

for feature_name, feature_score in zip(genetic_data.drop('class', axis=1).columns,
                                       fs.feature_importances_):
    print(feature_name, '\t', feature_score)

('N0', '\t', 12.5)
('N1', '\t', -332.0)
('N2', '\t', -496.0)
('N3', '\t', -498.0)
('N4', '\t', -238.0)
('N5', '\t', -559.0)
('N6', '\t', -605.5)
('N7', '\t', -408.5)
('N8', '\t', -369.0)
('N9', '\t', -569.5)
('N10', '\t', -29.0)
('N11', '\t', -359.0)
('N12', '\t', -394.0)
('N13', '\t', -195.0)
('N14', '\t', -184.0)
('N15', '\t', -403.5)
('N16', '\t', -516.5)
('N17', '\t', -462.5)
('P1', '\t', 6286.5)
('P2', '\t', 6612.0)


## 参考资料
1. [特征选择算法-Relief](http://blog.csdn.net/ferrarild/article/details/18792613)
2. [浅谈特征选择算法与Relief的实现](http://www.bubuko.com/infodetail-2156868.html)
3. [特征选择之relief及reliefF算法](http://blog.csdn.net/littlely_ll/article/details/71614826)
4. [ReliefF](https://github.com/EpistasisLab/scikit-rebate)
5. [特征选择文档](http://scikit-learn.org/stable/modules/feature_selection.html)